<a href="https://colab.research.google.com/github/deregenboogm/NLP_coronavirus_project/blob/master/text-summarization/summarizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
def generate_summary(text):
  ## Data pre-processing
  ### Removing square brackets and extra spaces
  text = re.sub(r'\[[0-9]*\]', ' ', text)
  text = re.sub(r'\s+', ' ', text)

  ### Removing special characters and digits
  cleaned_text = re.sub('[^a-zA-Z]', ' ', text)
  cleaned_text = re.sub(r'\s+', ' ', cleaned_text)


  ## Define a function to create the word frequency table
  def word_frequency_table(cleaned_text):
    stop_words = set(stopwords.words("english"))
    ps = PorterStemmer()

    words = word_tokenize(cleaned_text)
    pos_tag_words = nltk.pos_tag(words)
    pos_tag_noun_verb = []
    tags = ["NN", "NNP", "NNS", "VB", "VBD", "VBG", "VBN", "VBP", "VBZ"]
    for word,tag in pos_tag_words:
        if tag in tags:
          pos_tag_noun_verb.append(word)        
    
    frequency_table = {}
    for word in pos_tag_noun_verb:
      if word not in stop_words:
        word = word = ps.stem(word)
        if word in frequency_table:
            frequency_table[word] += 1
        else:
            frequency_table[word] = 1 
    
    return frequency_table

  ## Create the word frequency table
  frequency_table= word_frequency_table(cleaned_text) 

  
  ## Tokenize the sentences: Split the text in a list of sentences
  list_of_sentences = sent_tokenize(text)


  ## Define a function to create a sentence score table
  def score_sentences(list_of_sentences, frequency_table):    
    sentence_score_table = {}
    for sentence in list_of_sentences:
      number_of_words_in_sentence = len(sentence.split())
      if number_of_words_in_sentence > 2 and number_of_words_in_sentence < 70:
        for word in word_tokenize(sentence.lower()):          
          for word in frequency_table:
            if word in sentence.lower():                            
              if sentence in sentence_score_table:                
                sentence_score_table[sentence] += frequency_table[word] / number_of_words_in_sentence
              else:
                sentence_score_table[sentence] = frequency_table[word] / number_of_words_in_sentence
    return sentence_score_table


  ## Create a sentence score table 
  sentence_score_table = score_sentences(list_of_sentences, frequency_table)
  

  ## Define a function to create and limit the length of the summary
  def summary(list_of_sentences, sentence_score_table):
    # Sort sentences accoring to their values
    sentences_sorted = dict(sorted(sentence_score_table.items(), key=operator.itemgetter(1),reverse=True))
    sentences_sorted = list(sentences_sorted.keys())

    # Limit the number of summary sentences with the highest scores
    result = sentences_sorted[:3]
           
    # Collect the highest scored sentences
    candiates_for_summary = ""  
    for sentence in sentences_sorted:
      if sentence in  result:
        candiates_for_summary += " "+ sentence
    
    # Get the sentences in their original order in the text
    summary = ""
    for sentence in list_of_sentences:
      if sentence in candiates_for_summary:
          summary += " " + sentence  
         
    
    print("Number of words-- summary vs text: ", len(summary.split()), '/', len(text.split()))
    print("Number of sentences-- summary vs text: ", len(sent_tokenize(summary)), '/', len(sent_tokenize(text)))
    return summary
     
  ## Execute the function to generate the summarization of the given text
  summary = summary(list_of_sentences, sentence_score_table)

  return summary
